In [ ]:
# Importing Libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy as sp
import numpy as np
import pandas as pd
import time
import datetime
import os
import sys
import pyvisa
import threading
import ipywidgets
import nidaqmx
from scipy import signal
from collections import deque
from IPython.display import clear_output, display

print(f"All Libraries imported successfully at {datetime.datetime.now()}")

In [ ]:
# Connecting to Instruments
rm = pyvisa.ResourceManager()
ami430 = rm.open_resource('TCPIP::192.168.1.30::7180::SOCKET', timeout=5000, write_termination='\n', read_termination='\n')

print(f"Instruments connected successfully at {datetime.datetime.now()}")

In [ ]:
# Function for defining the stop button click event handler
def on_stop_clicked(b):
    stop_event.set()
    status_btn.description = "Stopped"
    status_btn.button_style = "danger"
    stop_btn.disabled = True
    stop_btn.close()

# Function for creating path for saving file in a specific folder
def save_file(folder,filename):
    """It will create the folder if it does not exist and return the full path for saving file in that folder."""
    current_path = os.getcwd()
    folder_path = os.path.join(current_path, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    filename = f"{datetime.datetime.now().strftime('%y%m%d')}_{filename}"
    file_path = os.path.join(folder_path, filename)
    return file_path

# Function for creating path for loading file from a specific folder
def load_file(folder,filename):
    """It will return the full path for loading file from that folder."""
    current_path = os.getcwd()
    if folder == str(datetime.datetime.now().strftime('%y%m%d')):
        filename = f"{datetime.datetime.now().strftime('%y%m%d')}_{filename}"
    else:
        filename = f"{folder}_{filename}"
    file_path = os.path.join(current_path, folder, filename)
    return file_path

# Function for creating array for sweeping
def sweep(vmin, vmax, vstep, vmode): # Modes: up, down, updown, downup, round-up, round-down
    if vmode =='up': # lower to higher value
        sweep = np.arange(vmin,vmax+vstep,vstep)
    elif vmode =='down': # higher to lower value
        sweep = np.arange(vmax,vmin-vstep,-vstep)
    elif vmode =='updown': # lower to higher to lower value
        sweep = np.arange(vmin,vmax+vstep,vstep)
        sweep = np.append(sweep,np.arange(vmax-vstep,vmin-vstep,-vstep))
    elif vmode =='downup': # higher to lower to higher value
        sweep = np.arange(vmax,vmin-vstep,-vstep)
        sweep = np.append(sweep,np.arange(vmin+vstep,vmax+vstep,vstep))
    elif vmode =='round-up': # zero to higher to lower to zero value
        sweep = np.arange(0,vmax+vstep,vstep)
        sweep = np.append(sweep,np.arange(vmax-vstep,vmin-vstep,-vstep))
        sweep = np.append(sweep,np.arange(vmin+vstep,vstep,vstep))
    elif vmode =='round-down': # zero to lower to higher to zero value
        sweep = np.arange(0,vmin-vstep,-vstep)
        sweep = np.append(sweep,np.arange(vmin+vstep,vmax+vstep,vstep))
        sweep = np.append(sweep,np.arange(vmax-vstep,-vstep,-vstep))
    else:
        print('Error: Invalid Sweep Mode')
        sys.exit()
    return sweep

# Function to set AMI430 field and ramp rate units
def set_ami_unit(field_unit, rate_unit, ami430):
    '''
    Tesla or kiloGauss for field_unit. per_minute or per_second for rate_unit.
    1 Tesla = 10 kiloGauss
    '''
    try:
        if field_unit == 'Tesla':
            field_unit_code = 1
        elif field_unit == 'kiloGauss':
            field_unit_code = 0
        if rate_unit == 'per_minute':
            rate_unit_code = 1
        elif rate_unit == 'per_second':
            rate_unit_code = 0
        ami430.write(f"CONFigure:FIELD:UNITS {field_unit_code}")
        ami430.write(f"CONFigure:RAMP:RATE:UNITS {rate_unit_code}")
        print(f'Units set to {field_unit} {rate_unit}')
    except:
        print('Error: Unable to set units. Check AMI430 and retry.')
        sys.exit()

# Function to get AMI430 status
def get_ami_status(ami430):
    ami_state_number = ami430.query_ascii_values('STATE?')
    ami_state_list = ["RAMPING to target", 
                      "HOLDING at target", 
                      "PAUSED", 
                      "Ramping in MANUAL UP mode", 
                      "Ramping in MANUAL DOWN mode", 
                      "ZEROING CURRENT", 
                      "QUENCH DETECTED", 
                      "AT ZERO current", 
                      "Heating persistent switch", 
                      "Cooling persistent switch"]
    ami_state = ami_state_list[int(ami_state_number[0])-1]
    return ami_state

# Function to set AMI430 magnetic field
def set_ami_field(B_set, ami430):
    '''arguments: B_set - target magnetic field, ami430 - AMI430 instrument object'''
    ami_state_number = ami430.query_ascii_values('STATE?')[0]
    if ami_state_number in [1, 2, 3]:
        ami430.write(f'CONFigure:FIELD:TARget {B_set}')
        ami430.write(f'RAMP')

# Function to set AMI430 ramp rate
def set_ami_ramp_rate(rate, field_limit, ami430):
    ami430.write('CONFigure:RAMP:RATE:SEGment 1')
    ami430.write(f'CONFigure:RAMP:RATE:FIELD 1,{rate},{field_limit}')

# Function to read temperature from lakeshore via gpib
def readtemp_lakeshore(connection,address, channel):
    if connection == 'GPIB':
        try:
            temp = address.query_ascii_values(f"RDGK? {channel}")[0]
        except Exception:
            temp = np.nan
    if connection == 'LOG':
        if address == 'fincryo': # For Fincryo
            folder="\\\DESKTOP-DQE01NH\Logfiles\\"+str(datetime.now().strftime("%y-%m-%d"))
            file=f"CH{int(channel)} T "+str(datetime.now().strftime("%y-%m-%d"))+".log"
        with open(os.path.join(folder, file), "rb") as f:
            f.seek(-2, 2)  # go to the end of file
            while f.read(1) != b"\n":
                f.seek(-2, 1)
            last_line = f.readline().decode().strip()
        temperature = float(last_line.split(',')[2])
    return temp

# Functions to set heater parameters
def set_heater(connection,address,mode, range, P, I, D):
    '''
    Set heater parameters:
    mode: Heater mode - "Closed Loop PID","Zone Tuning","Open Loop","Off"
    range: Heater range - "off","31.6 uA","100 uA","316 uA","1.00 mA","3.16 mA","10.0 mA","31.6 mA","100 mA"
    P, I, D: PID parameters (only for Closed Loop PID mode)
    '''
    if connection == 'GPIB':
        heater_range_list = ["off","31.6 uA","100 uA","316 uA","1.00 mA","3.16 mA","10.0 mA","31.6 mA","100 mA"]
        heater_mode_list = ["Closed Loop PID","Zone Tuning","Open Loop","Off"]
        if mode in heater_mode_list:
            mode_index = heater_mode_list.index(mode) + 1
            address.write(f'CMODE {mode_index}')
        else:
            print('Error: Invalid Heater Mode')
            sys.exit()
        if mode == "Closed Loop PID":
            if range in heater_range_list:
                range_index = heater_range_list.index(range)
                address.write(f'HTRRNG {range_index}')
            else:
                print('Error: Invalid Heater Range')
                sys.exit()
            address.write(f'PID {P},{I},{D}')
        print('Heater settings updated successfully')

# Function to get heater status        
def get_heater_status(connection,address):
    if connection == 'GPIB':
        # Function to read heater mode
        def read_heater_mode(address):
            n = address.query_ascii_values('CMODE?')[0]
            heater_mode_list = ["Closed Loop PID","Zone Tuning","Open Loop","Off"]
            val = heater_mode_list[int(n)-1]
            return val
        # Function to read heater range
        def read_heater_range(address):
            n = address.query_ascii_values('HTRRNG?')[0]
            heater_range_list = ["off","31.6 uA","100 uA","316 uA","1.00 mA","3.16 mA","10.0 mA","31.6 mA","100 mA"]
            val = heater_range_list[int(n)]
            return val
        # Function to read PID parameters
        def read_pid_params(address):
            params = address.query_ascii_values('PID?')
            return params
        # Reading all heater status
        mode = read_heater_mode(address)
        range = read_heater_range(address)
        P, I, D = read_pid_params(address)
        status = {
            "Mode": mode,
            "Range": range,
            "P": P,
            "I": I,
            "D": D
        }
    return status
print(f"All Helper functions are defined successfully at {datetime.datetime.now()}")

In [ ]:
# Program for logging AMI magnet parameters
stop_event = threading.Event()
status_btn = ipywidgets.Button(description="Running", button_style="success", disabled=True)
stop_btn = ipywidgets.Button(description="Stop measurement", button_style="danger")
display(status_btn, stop_btn)
stop_btn.on_click(on_stop_clicked)
# Measurement Loop in a separate thread
def ami_logging_loop():
    # Log file setup
    logfile = save_file("Magnet Log", "measurement_file_name.dat")
    if not os.path.exists(logfile):
        cols = ["datetime", "State", "Temperature", "Field", "Vmagnet", "Vsupply"]
        pd.DataFrame(columns=cols).to_csv(logfile, index=False)
    # Logging and plotting loop setup
    while not stop_event.is_set():
        # Data recording and saving to file
        t = datetime.datetime.now().strftime('%y-%m-%d %H:%M:%S')
        ami_state_number = ami430.query_ascii_values('STATE?')[0]
        Temp = np.random.randint(0, 100)
        Field = ami430.query_ascii_values('FIELD:MAGnet?')[0]
        V_magnet = ami430.query_ascii_values('VOLTage:MAGnet?')[0]
        V_supply = ami430.query_ascii_values('VOLTage:SUPPly?')[0]
        row = {"datetime": t, "State": ami_state_number, "Temperature": Temp, "Field": Field, "Vmagnet": V_magnet, "Vsupply": V_supply}
        pd.DataFrame([row]).to_csv(logfile, mode='a', header=False, index=False)
        time.sleep(1)

threading.Thread(target=ami_logging_loop, daemon=True).start()

In [ ]:
# Program for live plotting AMI magnet log
def live_plot_loop(folder, filename):
    time.sleep(1)
    logfile = load_file(folder, filename)  # uses your existing load_file()
    while not stop_event.is_set():
        df = pd.read_csv(logfile)
        df['datetime'] = pd.to_datetime(df['datetime'], format='%y-%m-%d %H:%M:%S', errors='coerce')
        df = df.dropna(subset=['datetime'])
        clear_output(wait=True)
        fig, axes = plt.subplots(4, 1, figsize=(10, 10), sharex=True)
        axes[0].plot(df['datetime'], df['Temperature'], 'o', markersize=3)
        axes[0].set_ylabel('T (K)')
        axes[1].plot(df['datetime'], df['Field'], 'o', markersize=3)
        axes[1].set_ylabel('B (T)')
        axes[2].plot(df['datetime'], df['Vmagnet'], 'o', markersize=3)
        axes[2].set_ylabel('Vmagnet (V)')
        axes[3].plot(df['datetime'], df['Vsupply'], 'o', markersize=3)
        axes[3].set_ylabel('Vsupply (V)')
        axes[3].set_xlabel('datetime')
        axes[3].xaxis.set_major_locator(mdates.AutoDateLocator())
        axes[3].xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d\n%H:%M:%S'))
        State = df['State'].iloc[-1]
        fig.suptitle(f'State: {State}', y=0.98)
        for ax in axes:
            ax.grid(True)
        fig.autofmt_xdate()
        display(fig)
        plt.close(fig)
        time.sleep(1)

threading.Thread(target=live_plot_loop, args=("Magnet Log", "measurement_file_name.dat"), daemon=True).start()

In [ ]:
# Program for changing and logging magnetic field using AMI430

B_min = 0  # Minimum magnetic field in Tesla
B_max = 1  # Maximum magnetic field in Tesla
B_step = 0.1  # Magnetic field step in Tesla
B_ramp_rate = 0.002  # Ramp rate in Tesla per minute

# Configuring AMI430
set_ami_unit('Tesla', 'per_minute', ami430)

# Open log file and write header
logfile = save_file("Magnet Log", "measurement_file_name.dat")
if not os.path.exists(logfile):
    cols = ["datetime", "State", "Temperature", "Field", "Vmagnet", "Vsupply"]
    pd.DataFrame(columns=cols).to_csv(logfile, index=False)

for B in sweep(B_min, B_max, B_step, 'up'):
    set_ami_ramp_rate(0.002, B_max, ami430)
    set_ami_field(B, ami430)
    while True:
        ami_state_number = ami430.query_ascii_values('STATE?')[0]
        if ami_state_number == 1:
            t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            ami_state_number = ami430.query_ascii_values('STATE?')[0]
            Temp = readtemp_lakeshore_gpib(lakeshore, 3)
            Field = ami430.query_ascii_values('FIELD:MAGnet?')[0]
            V_magnet = ami430.query_ascii_values('VOLTage:MAGnet?')[0]
            V_supply = ami430.query_ascii_values('VOLTage:SUPPly?')[0]
            row = {"datetime": t, "State": ami_state_number, "Temperature": Temp, "Field": Field, "Vmagnet": V_magnet, "Vsupply": V_supply}
            pd.DataFrame([row]).to_csv(logfile, mode='a', header=False, index=False)
        elif ami_state_number == 2:
            break
        elif ami_state_number == 7:
            print('Quench detected! Stopping measurement.')
            sys.exit()
